# Εργαστήριο 6
Όπως και στα προηγούμενα εργαστήρια συνεχίζουμε στο περιβάλλον του [online chisel bootcamp](https://mybinder.org/v2/gh/freechipsproject/chisel-bootcamp/master).

Πριν ξεκινήσετε, εκτελέστε τα επόμενα 2 κελιά:

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## 1. Επίτρεψη εγγραφής (enable) σε καταχωρητές
Στα παραδείγματα με καταχωρητές που είδαμε ως τώρα, σε κάθε νέο βήμα (step) του ρολογιού (clock), ο καταχωρητής αποθηκεύει ό,τι βρίσκεται στην είσοδό του.

Σε έναν επεξεργαστή συχνά η εγγραφή σε καταχωρητές δεν θέλουμε να γίνεται σε κάθε βήμα του ρολογιού, παρά μόνο αν επιτρέπεται από κάποια συνθήκη.

Στην Chisel μπορούμε πολύ εύκολα να επιλέξουμε πότε θα γίνει εγγραφή δεδομένων σε έναν καταχωρητή χρησιμοποιώντας τη δομή `when`. Στο παράδειγμα που ακολουθεί, τα δεδομένα εισόδου εγγράφονται στον καταχωρητή μόνο αν το επιτρέπει το εξωτερικό σήμα `enable` (1 bit, 1 = επίτρεψη εγγραφής, 0 = όχι):

In [ ]:
class EnableExample(n: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(n.W))
    val enable = Input(UInt(1.W))
    val out = Output(UInt(n.W))
  })
    
  val testReg = Reg(UInt(n.W))
  
  when(io.enable===1.U) {
    testReg := io.in    
  }
    
  io.out := testReg
}

### 1.1. Όταν δεν προσδιορίζεται η είσοδος σε καταχωρητή
Στο προηγούμενο παράδειγμα με τον καταχωρητή, παρατηρήστε ότι **δεν προσδιορίζεται** η είσοδος στον `testReg`
όταν το `io.enable` δεν είναι `1.U`! Στην περίπτωση αυτή απλά **διατηρείται η τρέχουσα τιμή του καταχωρητή**
(όποια κι αν είναι αυτή).


Δοκιμάστε το προηγούμενο κύκλωμα εκτελώντας το επόμενο κελί:

In [ ]:
test(new EnableExample(8)) { c =>
  c.io.enable.poke(0.U)   // δεν επιτρέπεται η εγγραφή στον καταχωρητή
  c.io.in.poke(33.U)
  c.clock.step()
  println(c.io.out.peek())    // το 33 δεν θα εμφανιστεί στην έξοδο του καταχωρητή
    
  c.io.enable.poke(1.U)    // επιτρέπεται η εγγραφή στο επόμενο clock
  c.clock.step()
  println(c.io.out.peek())    // το 33 θα εμφανιστεί στην έξοδο
}

## Άσκηση 1
Κατασκευάστε κύκλωμα με έναν καταχωρητή εύρους `n` bits (παραμετρική υλοποίηση) και τις αντίστοιχες εισόδους-εξόδους:
* `in` (εύρος n bits): είσοδος δεδομένων
* `out` (εύρος n bits): έξοδος δεδομένων

Σε κάθε βήμα (κύκλο) ρολογιού εισάγεται στο κύκλωμα μια νέα τιμή στην είσοδο αλλά **ο καταχωρητής συγκρατεί μόνο τιμές μεγαλύτερες από αυτήν που έχει ως αποθηκευμένη ως τώρα**. Συνεπώς ο καταχωρητής θα εμφανίζει κάθε στιγμή στην έξοδο **τη μέγιστη** από τις τιμές που έχουν εμφανιστεί ως τώρα.

Ο καταχωρητής θα πρέπει να είναι αρχικοποιημένος με την τιμή `0.U`.

Η Chisel διαθέτει τους εξής τελεστές σύγκρισης:

| τελεστές | έλεγχος |
| --- | --- |
| === | ισότητα |
| =/= | ανισότητα |
| >, >=, <, <= | σύγκριση |

Συμπληρώστε το επόμενο κελί.

In [ ]:
class MaxRegister(n: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(n.W))
    val out = Output(UInt(n.W))
  })
    
  // ..συμπληρώστε..
    
}

Δοκιμάστε τη λειτουργία του κυκλώματός σας με τον παρακάτω κώδικα ελέγχου: 

In [ ]:
test(new MaxRegister(8)) { c =>
  val numbers = List(10,20,5,7,22,9,44,2,43)
  for (number <- numbers) {
    c.io.in.poke(number.U)
    c.clock.step()
  }
  println(c.io.out.peek())
  c.io.out.expect(numbers.max.U) // το numbers.max θα επιστρέψει 44

}
println("SUCCESS!!")

## 2. Συστοιχία καταχωρητών (register file)
Σε έναν επεξεργαστή οι εντολές μηχανής χρησιμοποιούν έναν αριθμό καταχωρητών για την αποθήκευση προσωρινών αποτελεσμάτων. Η *συστοιχία καταχωρητών* (register file) είναι στην ουσία ένα «array» από καταχωρητές της Chisel.

Για να δημιουργήσουμε στην Chisel μια συστοιχία καταχωρητών χρησιμοποιούμε τη σύνταξη:
~~~scala
val regFile = Reg(Vec(m,UInt(n.W)))
~~~
Η πιο πάνω εντολή θα δημιουργήσει `m` καταχωρητές, με εύρος λέξης `n` ο καθένας, π.χ. για να δημιουργήσουμε 8 καταχωρητές με εύρος λέξης 32 bits θα γράψουμε:
~~~scala
val regFile = Reg(Vec(8,UInt(32.W)))
~~~

Αν θέλουμε οι καταχωρητές να έχουν **συγκεκριμένη αρχική τιμή**, η σύνταξη είναι λίγο πιο σύνθετη. Για `m` καταχωρητές, με εύρος λέξης `n` ο καθένας και αρχική τιμή `k` σε όλους γράφουμε:
~~~scala
val regFile = RegInit(VecInit(Seq.fill(m)(k.U(n.W))))
~~~

Για παράδειγμα, αν θέλουμε 8 καταχωρητές με εύρος λέξης 32 bits αρχικοποιημένους στο 0, θα γράψουμε:
~~~scala
val regFile = RegInit(VecInit(Seq.fill(8)(0.U(32.W))))
~~~
*Σημ.: η έκφραση `Seq.fill(8)(0.U(n.W))` είναι της υποκείμενης γλώσσας (Scala) και δημιουργεί μια λίστα 8 στοιχείων με τιμή `0.U(32.W)` το καθένα.*

Όταν το `regFile` οριστεί όπως στα δύο προηγούμενα, τότε μπορούμε να συνδέσουμε τον **i-οστό καταχωρητή** της συστοιχίας όπως στα επόμενα παραδείγματα:
~~~scala
x := regFile(i) // σύνδεση του x στην έξοδο του i-οστού καταχωρητή του regFile

regFile(i) := y // σύνδεση της εισόδου του i-οστού καταχωρητή του regFile στο y
~~~


### 2.1 Παράδειγμα register file
Δείτε το επόμενο παράδειγμα υλοποίησης ενός module με register file. Υπάρχουν `m` καταχωρητές, ενώ το εύρος λέξης είναι `n`. Η τιμή αρχικοποίησης των καταχωρητών είναι `0`.

Περιγραφή εισόδων-εξόδων:
* in (n bits): είσοδος δεδομένων
* out (n bits): έξοδος δεδομένων
* read_select (log2(m) bits): επιλογή καταχωρητή για ανάγνωση, τα δεδομένα του εμφανίζονται στην έξοδο
* write_select (log2(m) bits): επιλογή καταχωρητή για εγγραφή, τα δεδομένα εισόδου θα εγγραφούν σε αυτόν στο επόμενο βήμα ρολογιού.

*Σημ.: τα σήματα select έχουν εύρος log2(m) bits με στρογγυλοποίηση προς τα πάνω (συνάρτηση `log2Ceil()`) για να εξασφαλιστεί ότι «χωράνε» τον αριθμό κάθε καταχωρητή (0...m-1).*

In [ ]:
class SimpleRegisterFile(m: Int, n: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(n.W))
    val read_select = Input(UInt(log2Ceil(m).W))
    val write_select = Input(UInt(log2Ceil(m).W))      
    val out = Output(UInt(n.W))
  })
    
  val regFile = RegInit(VecInit(Seq.fill(m)(0.U(n.W))))
    
  regFile(io.write_select) := io.in    // εγγραφή δεδομένων στον καταχωρητή που επιλέγει το io.write_select
    
  io.out := regFile(io.read_select)    // ανάγνωση δεδομένων από τον καταχωρητή που επιλέγει το io.read_select
  
}

Στο επόμενο κελί δημιουργούμε μαι συστοιχία 8 καταχωρητών με μήκος λέξης 8 bits. Στη συνέχεια γράφουμε δοκιμαστικές τιμές σε καθένα από τους καταχωρητές (r0...r7) και στη συνέχεια τις τυπώνουμε με την ίδια σειρά.

In [ ]:
test(new SimpleRegisterFile(8,8)) { c =>
  // εγγραφή δεδομένων, r0 = 33, r1 = 34, r2 = 35 κλπ
  for (i <- 0 to 7) {
    c.io.write_select.poke(i.U)
    c.io.in.poke((33+i).U)
    c.clock.step()
  }
  // ανάγνωση δεδομένων από όλους τους καταχωρητές    
  for (i <- 0 to 7) {
    c.io.read_select.poke(i.U)
    println(i,c.io.out.peek())
  }

}

## Άσκηση 2
Τροποποιήστε το προηγούμενο παράδειγμα, έτσι ώστε η εγγραφή να πραγματοποιείται μόνο όταν το `write_select` είναι διάφορο του 0. Ο καταχωρητής 0 δηλαδή, **δεν εγγράφεται αλλά διατηρεί πάντα την αρχική του τιμή (0)**.

Συμπληρώστε στο επόμενο κελί:

In [ ]:
class RegisterFileZeroReg(m: Int, n: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(n.W))
    val read_select = Input(UInt(log2Ceil(m).W))
    val write_select = Input(UInt(log2Ceil(m).W))      
    val out = Output(UInt(n.W))
  })
    
  // ..συμπληρώστε..
    
}

Δοκιμάστε την ορθότητα της υλοποίησής σας με τον επόμενο κώδικα ελέγχου:

In [ ]:
test(new RegisterFileZeroReg(8,8)) { c =>

  for (i <- 0 to 7) {
    c.io.write_select.poke(i.U)
    c.io.in.poke((33+i).U)
    c.clock.step()
  }
    
  for (i <- 0 to 7) {
    c.io.read_select.poke(i.U)
    c.io.out.expect( if (i>0) (33+i).U else 0.U)   // εδώ ο r0 θα πρέπει να είναι 0 κι όχι 33
  }

}
println("SUCCESS!!")

## Τελειώνοντας...
Μπορείτε να κατεβάσετε και να πάρετε μαζί σας το σημερινό notebook για να το έχετε στο αρχείο σας.

**Δεν ανήκει στα παραδοτέα του εργαστηρίου.**